<a href="https://colab.research.google.com/github/AnuragSingh7292/Remote-sensing-image-retrieval-deep-learning-/blob/main/MatricesPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive in Colab

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# After running, follow the authentication steps


Mounted at /content/drive


Read Feature Vectors from Drive

In [ ]:
import numpy as np

feature_vector_path = "/content/drive/MyDrive/VGG16_UCMerced_landuse_features.npy"
features = np.load(feature_vector_path)

print("Feature vector shape:", features.shape)


Feature vector shape: (2100, 512)


Step 1: Load Feature Vectors and Ground Truth

In [ ]:
import numpy as np

# Load your feature vectors (e.g., from a .npy file or any other source)
# Example: features = np.load('features.npy')
# Ensure features is a 2D numpy array of shape (num_images, feature_dimension)

# Load your labels corresponding to each feature vector
# Example: labels = np.load('labels.npy')
# Ensure labels is a 1D numpy array of shape (num_images,)


Extract the ZIP File

In [ ]:
import zipfile
import os

# Define the paths
zip_path = "/content/drive/MyDrive/Images.zip"  # Update with your actual file path
extract_path = "/content/UC_Merced_LandUse"  # Temporary extraction path

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
print("Extracted categories:", os.listdir(extract_path))  # Should list 21 folders


Extracted categories: ['Images']


In [ ]:
extract_path = "/content/UC_Merced_LandUse/Images"
# Verify extraction
print("Extracted categories:", os.listdir(extract_path))  # Should list 21 folders

Extracted categories: ['freeway', 'agricultural', 'denseresidential', 'forest', 'overpass', 'storagetanks', 'runway', 'mediumresidential', 'tenniscourt', 'chaparral', 'baseballdiamond', 'intersection', 'airplane', 'mobilehomepark', 'buildings', 'sparseresidential', 'beach', 'harbor', 'golfcourse', 'parkinglot', 'river']


Code to Generate Labels from Folder Structure

In [ ]:
import numpy as np

# Get sorted list of category folders
categories = sorted(os.listdir(extract_path))
category_to_label = {category: idx for idx, category in enumerate(categories)}

# Initialize lists to store image paths and labels
image_paths = []
labels = []

# Iterate over each category folder
for category in categories:
    category_path = os.path.join(extract_path, category)
    if os.path.isdir(category_path):  # Ensure it's a directory
        for image_name in sorted(os.listdir(category_path)):  # Ensure sorted order
            image_paths.append(os.path.join(category_path, image_name))
            labels.append(category_to_label[category])

# Convert labels to NumPy array
labels = np.array(labels)

# Print some sample results
print(f"Total images: {len(image_paths)}")
print(f"Total labels: {len(labels)}")
print(f"Categories: {categories}")
print(f"First 10 labels: {labels[:10]}")


Total images: 2100
Total labels: 2100
Categories: ['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings', 'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse', 'harbor', 'intersection', 'mediumresidential', 'mobilehomepark', 'overpass', 'parkinglot', 'river', 'runway', 'sparseresidential', 'storagetanks', 'tenniscourt']
First 10 labels: [0 0 0 0 0 0 0 0 0 0]


Save Labels for Future Use

In [ ]:
label_save_path = "/content/drive/MyDrive/labels.npy"
np.save(label_save_path, labels)
print(f"Labels saved at: {label_save_path}")

Labels saved at: /content/drive/MyDrive/labels.npy


Step 2: Compute Pairwise Distances

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity (higher = more similar)
similarity_matrix = cosine_similarity(features)

# Convert similarity to distance (lower = more similar)
distance_matrix = 1 - similarity_matrix
print(distance_matrix.shape)


(2100, 2100)


Step 3: Compute MAP

In [ ]:
def compute_map(labels, distance_matrix):
    num_images = len(labels)
    average_precisions = []

    for i in range(num_images):
        # Sort indices based on distance (ascending order)
        sorted_indices = np.argsort(distance_matrix[i])
        # Exclude the query image itself
        sorted_indices = sorted_indices[sorted_indices != i]
        # Retrieve the sorted labels
        sorted_labels = labels[sorted_indices]

        # Compute precision at each rank
        relevant_items = (sorted_labels == labels[i])
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)
        average_precision = np.sum(precision_at_k * relevant_items) / np.sum(relevant_items)
        average_precisions.append(average_precision)

    # Mean of average precisions
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

map_score = compute_map(labels, distance_matrix)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


Mean Average Precision (MAP): 0.0777


Step 4: Compute ANMRR

In [ ]:
def compute_anmrr(labels, distance_matrix, K=20):
    num_images = len(labels)
    anmrr_values = []

    for i in range(num_images):
        # Sort indices based on distance (ascending order)
        sorted_indices = np.argsort(distance_matrix[i])
        # Exclude the query image itself
        sorted_indices = sorted_indices[sorted_indices != i]
        # Retrieve the sorted labels
        sorted_labels = labels[sorted_indices]

        # Identify relevant items
        relevant_items = (sorted_labels == labels[i])
        num_relevant = np.sum(relevant_items)

        if num_relevant == 0:
            continue

        # Compute rank positions of relevant items
        rank_positions = np.where(relevant_items)[0] + 1  # +1 for 1-based indexing

        # Compute Modified Retrieval Rank (MRR)
        mrr = np.sum(rank_positions) / num_relevant

        # Compute Normalized Modified Retrieval Rank (NMRR)
        nmrr = (mrr - 0.5 * (1 + num_relevant)) / (1.25 * K - 0.5 * (1 + num_relevant))

        anmrr_values.append(nmrr)

    # Mean of NMRR values
    anmrr = np.mean(anmrr_values)
    return anmrr

anmrr_score = compute_anmrr(labels, distance_matrix)
print(f"Average Normalized Modified Retrieval Rank (ANMRR): {anmrr_score:.4f}")


Average Normalized Modified Retrieval Rank (ANMRR): -39.5353


In [ ]:
import numpy as np

def compute_map(labels, distance_matrix):
    num_images = len(labels)
    average_precisions = []

    for i in range(num_images):
        # Get sorted indices (excluding the query itself)
        sorted_indices = np.argsort(distance_matrix[i])
        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude self-match

        # Retrieve the corresponding sorted labels
        sorted_labels = labels[sorted_indices]

        # Compute relevant items (1 if label matches, else 0)
        relevant_items = (sorted_labels == labels[i]).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Mean of Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute MAP
map_score = compute_map(labels, distance_matrix)
print(f"Mean Average Precision (MAP): {map_score * 100:.2f}%")  # Convert to %


Mean Average Precision (MAP): 7.77%


In [ ]:
print("Unique labels:", np.unique(labels))
print("Number of images per label:", {l: sum(labels == l) for l in np.unique(labels)})


Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Number of images per label: {0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100, 7: 100, 8: 100, 9: 100, 10: 100, 11: 100, 12: 100, 13: 100, 14: 100, 15: 100, 16: 100, 17: 100, 18: 100, 19: 100, 20: 100}


In [ ]:
import numpy as np

def compute_map(labels, distance_matrix):
    num_images = len(labels)
    average_precisions = []

    for i in range(num_images):
        # Sort indices based on distance (ascending for Euclidean, descending for similarity)
        sorted_indices = np.argsort(distance_matrix[i])  # Works for Euclidean distance

        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude query itself
        sorted_labels = labels[sorted_indices]  # Get sorted labels

        # Compute relevant items (1 if label matches, else 0)
        relevant_items = (sorted_labels == labels[i]).astype(int)
        num_relevant = np.sum(relevant_items)

        # Avoid division by zero
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute MAP
map_score = compute_map(labels, distance_matrix)
print(f"Mean Average Precision (MAP): {map_score * 100:.2f}%")  # Convert to %


Mean Average Precision (MAP): 7.77%


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity (higher = more similar)
similarity_matrix = cosine_similarity(features)

# Convert similarity to distance (lower = more similar)
distance_matrix = 1 - similarity_matrix

# Check distances
print("Distance matrix shape:", distance_matrix.shape)
print("First row of distance matrix:", distance_matrix[0][:10])


Distance matrix shape: (2100, 2100)
First row of distance matrix: [5.9604645e-08 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]


In [ ]:


import scipy.io

# Load the .mat file
# mat_data = scipy.io.loadmat("/content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer_VLAD.mat")

# Print keys to check what’s inside
print("Keys in .mat file:", mat_data.keys())


# Assuming the key is 'features' (replace if different)
features = mat_data['vlad_features']

print("Feature vectors shape:", features.shape)
print("First feature vector:", features[0][:10])  # Print first 10 values
print("Mean of features:", np.mean(features))
print("Standard deviation of features:", np.std(features))

num_zero_vectors = np.sum(np.all(features == 0, axis=1))
print(f"Number of zero vectors: {num_zero_vectors} out of {features.shape[0]}")



Keys in .mat file: dict_keys(['__header__', '__version__', '__globals__', 'vlad_features'])
Feature vectors shape: (2100, 128)
First feature vector: [-3.0592380e-02  8.4967002e-02 -5.5749144e-04 -1.7726763e-01
  1.4663111e-07 -1.3210106e-02 -2.1063549e-02  3.2991997e-07
 -1.2447587e-02  0.0000000e+00]
Mean of features: -0.0020411666
Standard deviation of features: 0.08836478
Number of zero vectors: 0 out of 2100


In [ ]:
from sklearn.preprocessing import normalize

# L2 normalize feature vectors
features = normalize(features)
print(features.shape)

print("First normalized feature vector:", features[0][:10])
print("Mean after normalization:", np.mean(features))
print("Standard deviation after normalization:", np.std(features))


(2100, 128)
First normalized feature vector: [-3.0592382e-02  8.4967010e-02 -5.5749150e-04 -1.7726764e-01
  1.4663112e-07 -1.3210107e-02 -2.1063551e-02  3.2992000e-07
 -1.2447588e-02  0.0000000e+00]
Mean after normalization: -0.0020411666
Standard deviation after normalization: 0.08836478


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

mat_data = scipy.io.loadmat("/content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer_VLAD.mat")

# Print keys to check what’s inside
print("Keys in .mat file:", mat_data.keys())


# Assuming the key is 'features' (replace if different)
features11 = mat_data['vlad_features']
# Compute cosine similarity
similarity_matrix = cosine_similarity(features)

# Convert to distance (lower = more similar)
distance_matrix = 1 - similarity_matrix

# Check distances
print("Distance matrix shape:", distance_matrix.shape)
print("First row of distance matrix:", distance_matrix[0][:10])


Keys in .mat file: dict_keys(['__header__', '__version__', '__globals__', 'vlad_features'])
Distance matrix shape: (2100, 2100)
First row of distance matrix: [-1.1920929e-07  6.0075641e-02  1.9829988e-01  7.7331424e-02
  9.0671742e-01  1.7024088e+00  1.7331705e+00  7.1373117e-01
  1.0911895e+00  1.1684359e+00]


In [ ]:
# Fix negative values (force all distances to be non-negative)
distance_matrix = np.maximum(distance_matrix, 0)

print("Min distance after fix:", np.min(distance_matrix))
print("Max distance:", np.max(distance_matrix))
print("Mean distance:", np.mean(distance_matrix))
print("Standard deviation of distance:", np.std(distance_matrix))


Min distance after fix: 0.0
Max distance: 1.9719291
Mean distance: 0.99779963
Standard deviation of distance: 0.2981826


In [ ]:
def compute_map(labels, distance_matrix):
    num_images = len(labels)
    average_precisions = []

    for i in range(num_images):
        sorted_indices = np.argsort(distance_matrix[i])  # Sort by increasing distance
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute precision at each rank
        relevant_items = (sorted_labels == labels[i])
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)
        average_precision = np.sum(precision_at_k * relevant_items) / np.sum(relevant_items)
        average_precisions.append(average_precision)

    return np.mean(average_precisions)  # MAP score

# Compute MAP
map_score = compute_map(labels, distance_matrix)
print(f"Mean Average Precision (MAP): {map_score * 100:.2f}%")  # Convert to %


Mean Average Precision (MAP): 15.81%


In [ ]:
print("Feature vectors shape:", features.shape)
print("First feature vector:", features[0][:10])  # Print first 10 values
print("Mean of features:", np.mean(features))
print("Standard deviation of features:", np.std(features))


Feature vectors shape: (2100, 128)
First feature vector: [-3.0592382e-02  8.4967010e-02 -5.5749150e-04 -1.7726764e-01
  1.4663112e-07 -1.3210107e-02 -2.1063551e-02  3.2992000e-07
 -1.2447588e-02  0.0000000e+00]
Mean of features: -0.0020411666
Standard deviation of features: 0.08836478


In [ ]:
from sklearn.preprocessing import normalize

# Normalize feature vectors (L2 normalization)
features = normalize(features)

print("First normalized feature vector:", features[0][:10])


First normalized feature vector: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity matrix (higher = more similar)
similarity_matrix = cosine_similarity(features)

# Convert to distance (lower = more similar)
distance_matrix = 1 - similarity_matrix

# Check if distances are now reasonable
print("Distance matrix shape:", distance_matrix.shape)
print("First row of distance matrix:", distance_matrix[0][:10])


Distance matrix shape: (2100, 2100)
First row of distance matrix: [5.9604645e-08 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]


In [ ]:
import os
os.kill(os.getpid(), 9)


SyntaxError: invalid syntax (<ipython-input-1-4ea0069c65a4>, line 1)

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.preprocessing import image
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Path to dataset
dataset_path = "/content/UC_Merced_LandUse/Images"

# Save extracted features
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Image sizes
IMAGE_SIZES = {
    "alexnet": 224,
    "resnet50": 224,
    "vgg16": 224,
    "vgg19": 224,
    "inception": 299
}

# PyTorch models (AlexNet & ResNet50)
PYTORCH_MODELS = {
    "alexnet": models.alexnet(pretrained=True).eval(),
    "resnet50": models.resnet50(pretrained=True).eval(),
}

# Remove FC layer for ResNet50
PYTORCH_MODELS["resnet50"] = nn.Sequential(*list(PYTORCH_MODELS["resnet50"].children())[:-1])

# TensorFlow models (VGG16, VGG19, Inception)
TENSORFLOW_MODELS = {
    "vgg16": VGG16(weights="imagenet", include_top=False, pooling="avg"),
    "vgg19": VGG19(weights="imagenet", include_top=False, pooling="avg"),
    "inception": InceptionV3(weights="imagenet", include_top=False, pooling="avg")
}

# Preprocessing functions
PREPROCESS_FUNCS = {
    "vgg16": vgg16_preprocess,
    "vgg19": vgg19_preprocess,
    "inception": inception_preprocess
}

# Custom Dataset for PyTorch
class UCMercedDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        categories = sorted(os.listdir(root_dir))
        for idx, category in enumerate(categories):
            category_path = os.path.join(root_dir, category)
            if os.path.isdir(category_path):
                for img_file in os.listdir(category_path):
                    if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                        self.image_paths.append(os.path.join(category_path, img_file))
                        self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

# Extract features using PyTorch models
def extract_features_pytorch(model_name, model, image_size):
    print(f"\nExtracting features using {model_name.upper()} (PyTorch)...")

    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    dataset = UCMercedDataset(dataset_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    features = []
    labels = []

    with torch.no_grad():
        for imgs, lbls in dataloader:
            imgs = imgs.cuda() if torch.cuda.is_available() else imgs
            output = model(imgs)
            output = torch.flatten(output, start_dim=1)  # Flatten for storage
            features.append(output.cpu().numpy())
            labels.extend(lbls.numpy())

    features = np.vstack(features)
    labels = np.array(labels)

    np.save(os.path.join(save_dir, f"{model_name}_features.npy"), features)
    np.save(os.path.join(save_dir, f"{model_name}_labels.npy"), labels)

    print(f"Saved {model_name.upper()} features: {features.shape}")
    return features

# Extract features using TensorFlow models
def extract_features_tensorflow(model_name, model, image_size):
    print(f"\nExtracting features using {model_name.upper()} (TensorFlow)...")

    features = []
    labels = []

    categories = sorted(os.listdir(dataset_path))
    for idx, category in enumerate(categories):
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):
            for img_file in os.listdir(category_path):
                if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                    img_path = os.path.join(category_path, img_file)

                    # Load and preprocess image
                    img = image.load_img(img_path, target_size=(image_size, image_size))
                    img_array = image.img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)
                    img_array = PREPROCESS_FUNCS[model_name](img_array)

                    # Extract features
                    feature = model.predict(img_array)
                    features.append(feature.flatten())
                    labels.append(idx)

    features = np.array(features)
    labels = np.array(labels)

    np.save(os.path.join(save_dir, f"{model_name}_features.npy"), features)
    np.save(os.path.join(save_dir, f"{model_name}_labels.npy"), labels)

    print(f"Saved {model_name.upper()} features: {features.shape}")
    return features

# Run feature extraction
if __name__ == "__main__":
    extracted_features = {}

    # Extract PyTorch Features (AlexNet, ResNet50)
    for model_name, model in PYTORCH_MODELS.items():
        extracted_features[model_name] = extract_features_pytorch(model_name, model, IMAGE_SIZES[model_name])

    # Extract TensorFlow Features (VGG16, VGG19, Inception)
    for model_name, model in TENSORFLOW_MODELS.items():
        extracted_features[model_name] = extract_features_tensorflow(model_name, model, IMAGE_SIZES[model_name])

    print("\n✅ All feature vectors extracted and saved!")



Extracting features using ALEXNET (PyTorch)...


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

features  extraction for all model

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.preprocessing import image
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"

# Save extracted features
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Image sizes
IMAGE_SIZES = {
    "alexnet": 224,
    "resnet50": 224,
    "vgg16": 224,
    "vgg19": 224,
    "inception": 299
}

# Load PyTorch models
PYTORCH_MODELS = {
    "alexnet": models.alexnet(pretrained=True).eval(),
    "resnet50": models.resnet50(pretrained=True).eval(),
}

# Modify ResNet50 to remove FC layer
PYTORCH_MODELS["resnet50"] = nn.Sequential(*list(PYTORCH_MODELS["resnet50"].children())[:-1])

# Load TensorFlow models
TENSORFLOW_MODELS = {
    "vgg16": VGG16(weights="imagenet", include_top=False, pooling="avg"),
    "vgg19": VGG19(weights="imagenet", include_top=False, pooling="avg"),
    "inception": InceptionV3(weights="imagenet", include_top=False, pooling="avg")
}

# Preprocessing functions
PREPROCESS_FUNCS = {
    "vgg16": vgg16_preprocess,
    "vgg19": vgg19_preprocess,
    "inception": inception_preprocess
}

# Custom PyTorch dataset class
class UCMercedDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        categories = sorted(os.listdir(root_dir))
        for idx, category in enumerate(categories):
            category_path = os.path.join(root_dir, category)
            if os.path.isdir(category_path):
                for img_file in os.listdir(category_path):
                    if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                        self.image_paths.append(os.path.join(category_path, img_file))
                        self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

# Extract features using PyTorch
def extract_features_pytorch(model_name, model, image_size):
    print(f"\nExtracting features using {model_name.upper()} (PyTorch)...")
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = UCMercedDataset(dataset_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    features = []
    labels = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    with torch.no_grad():
        for imgs, lbls in dataloader:
            imgs = imgs.to(device)
            output = model(imgs)
            features.append(output.cpu().numpy().reshape(imgs.shape[0], -1))
            labels.extend(lbls.numpy())

    features = np.vstack(features)
    labels = np.array(labels)

    np.save(os.path.join(save_dir, f"{model_name}_features.npy"), features)
    np.save(os.path.join(save_dir, f"{model_name}_labels.npy"), labels)
    print(f"Saved {model_name.upper()} features: {features.shape}")
    return features

# Extract features using TensorFlow
def extract_features_tensorflow(model_name, model, image_size):
    print(f"\nExtracting features using {model_name.upper()} (TensorFlow)...")
    features = []
    labels = []

    categories = sorted(os.listdir(dataset_path))
    for idx, category in enumerate(categories):
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):
            for img_file in os.listdir(category_path):
                if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                    img_path = os.path.join(category_path, img_file)
                    img = image.load_img(img_path, target_size=(image_size, image_size))
                    img_array = image.img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)
                    img_array = PREPROCESS_FUNCS[model_name](img_array)
                    feature = model.predict(img_array)
                    features.append(feature.flatten())
                    labels.append(idx)

    features = np.array(features)
    labels = np.array(labels)
    np.save(os.path.join(save_dir, f"{model_name}_features.npy"), features)
    np.save(os.path.join(save_dir, f"{model_name}_labels.npy"), labels)
    print(f"Saved {model_name.upper()} features: {features.shape}")
    return features

# Main script
if __name__ == "__main__":
    extracted_features = {}
    num_images = sum([len(files) for _, _, files in os.walk(dataset_path)])
    print(f"Total images in dataset: {num_images}")

    for model_name, model in PYTORCH_MODELS.items():
        extracted_features[model_name] = extract_features_pytorch(model_name, model, IMAGE_SIZES[model_name])

    for model_name, model in TENSORFLOW_MODELS.items():
        extracted_features[model_name] = extract_features_tensorflow(model_name, model, IMAGE_SIZES[model_name])

    print("\n✅ All feature vectors extracted and saved!")

Total images in dataset: 2100

Extracting features using ALEXNET (PyTorch)...
Saved ALEXNET features: (2100, 1000)

Extracting features using RESNET50 (PyTorch)...
Saved RESNET50 features: (2100, 2048)

Extracting features using VGG16 (TensorFlow)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4

InceptionV3

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.preprocessing import image

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"

# Save extracted features
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Image size for InceptionV3
IMAGE_SIZE = 299

# Load InceptionV3 model (TensorFlow)
inception_model = InceptionV3(weights="imagenet", include_top=False, pooling="avg")

# Function to extract features using InceptionV3
def extract_features_inception(model, image_size):
    print(f"\nExtracting features using InceptionV3 (TensorFlow)...")
    features = []
    labels = []

    categories = sorted(os.listdir(dataset_path))
    for idx, category in enumerate(categories):
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):
            for img_file in os.listdir(category_path):
                if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                    img_path = os.path.join(category_path, img_file)
                    img = image.load_img(img_path, target_size=(image_size, image_size))
                    img_array = image.img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)
                    img_array = inception_preprocess(img_array)
                    feature = model.predict(img_array)
                    features.append(feature.flatten())
                    labels.append(idx)

    features = np.array(features)
    labels = np.array(labels)
    np.save(os.path.join(save_dir, "inception_features.npy"), features)
    np.save(os.path.join(save_dir, "inception_labels.npy"), labels)
    print(f"✅ Saved InceptionV3 features: {features.shape}")
    return features

# Extract InceptionV3 features
if __name__ == "__main__":
    extracted_features_inception = extract_features_inception(inception_model, IMAGE_SIZE)
    print("\n✅ InceptionV3 feature extraction completed and saved!")



Extracting features using InceptionV3 (TensorFlow)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/st

features Shape

In [ ]:
import os
import numpy as np

# Path where feature vectors are stored
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# List of models used
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Print the shape of feature vectors for each model
for model in models:
    feature_path = os.path.join(save_dir, f"{model}_features.npy")
    label_path = os.path.join(save_dir, f"{model}_labels.npy")

    if os.path.exists(feature_path):
        features = np.load(feature_path)
        labels = np.load(label_path)
        print(f"{model.upper()} Feature Shape: {features.shape}, Labels Shape: {labels.shape}")
    else:
        print(f"Feature file for {model} not found!")
import os
import numpy as np

# Path where feature vectors are stored
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# List of models used
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Print the shape of feature vectors for each model
for model in models:
    feature_path = os.path.join(save_dir, f"{model}_features.npy")
    label_path = os.path.join(save_dir, f"{model}_labels.npy")

    if os.path.exists(feature_path):
        features = np.load(feature_path)
        labels = np.load(label_path)
        print(f"{model.upper()} Feature Shape: {features.shape}, Labels Shape: {labels.shape}")
    else:
        print(f"Feature file for {model} not found!")


ALEXNET Feature Shape: (2100, 1000), Labels Shape: (2100,)
RESNET50 Feature Shape: (2100, 2048), Labels Shape: (2100,)
VGG16 Feature Shape: (2100, 512), Labels Shape: (2100,)
VGG19 Feature Shape: (2100, 512), Labels Shape: (2100,)
INCEPTION Feature Shape: (2100, 2048), Labels Shape: (2100,)
ALEXNET Feature Shape: (2100, 1000), Labels Shape: (2100,)
RESNET50 Feature Shape: (2100, 2048), Labels Shape: (2100,)
VGG16 Feature Shape: (2100, 512), Labels Shape: (2100,)
VGG19 Feature Shape: (2100, 512), Labels Shape: (2100,)
INCEPTION Feature Shape: (2100, 2048), Labels Shape: (2100,)


In [ ]:
from sklearn.decomposition import PCA

# Load saved features
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"

resnet101_fc6 = np.load(os.path.join(save_dir, "vgg16_features.npy"))

resnet152_fc6 = np.load(os.path.join(save_dir, "resnet152_fc6_32_features.npy"))

labels = np.load(os.path.join(save_dir, "labels.npy"))

pca_fc6 = PCA(n_components=32)
fc6_reduced = pca_fc6.fit_transform(fc6_features)

MAP

top_K = 10


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import average_precision_score

# Load extracted features and labels
def load_features(model_name, save_dir):
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")
    return features, labels

# Compute Mean Average Precision (MAP)
def compute_map(features, labels, top_k=10):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order)
        sorted_indices = np.argsort(-similarities)
        sorted_labels = labels[sorted_indices]

        # Remove self-match (first element)
        sorted_labels = sorted_labels[1:top_k + 1]

        # Compute precision scores
        relevant = (sorted_labels == query_label).astype(int)
        average_precision = average_precision_score(relevant, np.arange(1, top_k + 1))
        average_precisions.append(average_precision)

    map_score = np.mean(average_precisions)
    return map_score

# Directory where features are saved
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# Models list
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Compute and print MAP for each model
for model_name in models:
    features, labels = load_features(model_name, save_dir)
    map_score = compute_map(features, labels, top_k=10)
    print(f"MAP for {model_name.upper()}: {map_score:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

MAP for ALEXNET: 0.7307


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

MAP for RESNET50: 0.8264


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

MAP for VGG19: 0.8010


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

MAP for INCEPTION: 0.8007


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


Model	           MAP (%)

AlexNet	       73.07

ResNet50	   82.64

VGG16	         80.68

VGG19	         80.10

Inception	     80.07


top_k = 100

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import average_precision_score

# Load extracted features and labels
def load_features(model_name, save_dir):
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")
    return features, labels

# Compute Mean Average Precision (MAP)
def compute_map(features, labels, top_k=10):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order)
        sorted_indices = np.argsort(-similarities)
        sorted_labels = labels[sorted_indices]

        # Remove self-match (first element)
        sorted_labels = sorted_labels[1:top_k + 1]

        # Compute precision scores
        relevant = (sorted_labels == query_label).astype(int)
        average_precision = average_precision_score(relevant, np.arange(1, top_k + 1))
        average_precisions.append(average_precision)

    map_score = np.mean(average_precisions)
    return map_score

# Directory where features are saved
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# Models list
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Compute and print MAP for each model
for model_name in models:
    features, labels = load_features(model_name, save_dir)
    map_score = compute_map(features, labels, top_k=100)
    print(f"MAP for {model_name.upper()}: {map_score:.4f}")


MAP for ALEXNET: 0.3441
MAP for RESNET50: 0.4352
MAP for VGG16: 0.4135
MAP for VGG19: 0.4153


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


MAP for INCEPTION: 0.3841


In [ ]:
top_K = 50

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import average_precision_score

# Load extracted features and labels
def load_features(model_name, save_dir):
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")
    return features, labels

# Compute Mean Average Precision (MAP)
def compute_map(features, labels, top_k=10):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order)
        sorted_indices = np.argsort(-similarities)
        sorted_labels = labels[sorted_indices]

        # Remove self-match (first element)
        sorted_labels = sorted_labels[1:top_k + 1]

        # Compute precision scores
        relevant = (sorted_labels == query_label).astype(int)
        average_precision = average_precision_score(relevant, np.arange(1, top_k + 1))
        average_precisions.append(average_precision)

    map_score = np.mean(average_precisions)
    return map_score

# Directory where features are saved
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# Models list
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Compute and print MAP for each model
for model_name in models:
    features, labels = load_features(model_name, save_dir)
    map_score = compute_map(features, labels, top_k=50)
    print(f"MAP for {model_name.upper()}: {map_score:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


MAP for ALEXNET: 0.4811


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


MAP for RESNET50: 0.6061


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


MAP for VGG16: 0.5764
MAP for VGG19: 0.5741


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


MAP for INCEPTION: 0.5550


In [ ]:
top_k = 100


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import average_precision_score

# Load extracted features and labels
def load_features(model_name, save_dir):
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")
    return features, labels

# Compute Mean Average Precision (MAP)
def compute_map(features, labels, top_k=10):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order)
        sorted_indices = np.argsort(-similarities)
        sorted_labels = labels[sorted_indices]

        # Remove self-match (first element)
        sorted_labels = sorted_labels[1:top_k + 1]

        # Compute precision scores
        relevant = (sorted_labels == query_label).astype(int)
        average_precision = average_precision_score(relevant, np.arange(1, top_k + 1))
        average_precisions.append(average_precision)

    map_score = np.mean(average_precisions)
    return map_score

# Directory where features import numpy as np
from sklearn.metrics import average_precision_score

def mean_average_precision(y_true_list, y_score_list):
    """
    Computes the Mean Average Precision (MAP) for multiple queries.

    :param y_true_list: List of lists, where each sublist contains binary labels (1 for relevant, 0 for non-relevant).
    :param y_score_list: List of lists, where each sublist contains similarity scores for the retrieved images.
    :return: Mean Average Precision (MAP) score.
    """
    average_precisions = []

    for y_true, y_score in zip(y_true_list, y_score_list):
        if sum(y_true) == 0:  # Avoid division by zero
            continue

        ap = average_precision_score(y_true, y_score)
        average_precisions.append(ap)

    return np.mean(average_precisions) if average_precisions else 0.0

# Example usage:
# y_true_list: Each sublist contains ground truth (1 for relevant, 0 for non-relevant)
# y_score_list: Each sublist contains similarity scores

y_true_list = [
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 1],  # Example for one query
    [0, 1, 0, 1, 0, 0, 1, 1, 0, 1],  # Another query
    # Add more queries as needed...
]

y_score_list = [
    [0.9, 0.2, 0.8, 0.1, 0.75, 0.05, 0.04, 0.02, 0.7, 0.65],  # Scores for first query
    [0.1, 0.9, 0.15, 0.85, 0.05, 0.02, 0.8, 0.75, 0.03, 0.7],  # Scores for second query
    # Add more queries as needed...
]

# Compute MAP
map_score = mean_average_precision(y_true_list, y_score_list)
print(f"Mean Average Precision (MAP): {map_score:.4f}")

save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# Models list
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Compute and print MAP for each model
for model_name in models:
    features, labels = load_features(model_name, save_dir)
    map_score = compute_map(features, labels, top_k=100)
    print(f"MAP for {model_name.upper()}: {map_score:.4f}")


Mean Average Precision (MAP): 1.0000
MAP for ALEXNET: 0.3441
MAP for RESNET50: 0.4352
MAP for VGG16: 0.4135
MAP for VGG19: 0.4153


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


MAP for INCEPTION: 0.3841


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels
def load_features(model_name, save_dir):
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")
    return features, labels

# Compute Mean Average Precision (MAP) for all retrieved images
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Directory where features are saved
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# Models list
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Compute and print MAP for each model
for model_name in models:
    features, labels = load_features(model_name, save_dir)
    map_score = compute_map(features, labels)
    print(f"MAP for {model_name.upper()}: {map_score * 100:.2f}%")  # Convert to percentage


MAP for ALEXNET: 42.98%
MAP for RESNET50: 55.82%
MAP for VGG16: 53.07%
MAP for VGG19: 52.98%
MAP for INCEPTION: 49.14%


for TOP_K = 10

In [ ]:
import numpy as np

def compute_anmrr(labels, distance_matrix, k=100):
    """
    Compute the Average Normalized Modified Retrieval Rank (ANMRR).

    Parameters:
        labels (numpy array): Array of ground truth labels.
        distance_matrix (numpy array): Precomputed distance matrix (higher = less similar).
        k (int): The number of top retrieved images to consider.

    Returns:
        float: ANMRR score (lower is better).
    """
    num_images = len(labels)
    ranks = []

    for i in range(num_images):
        # Get sorted indices (excluding the query itself)
        sorted_indices = np.argsort(distance_matrix[i])
        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude self-match

        # Retrieve the corresponding sorted labels
        sorted_labels = labels[sorted_indices]

        # Find ranks of relevant items
        relevant_indices = np.where(sorted_labels == labels[i])[0]  # Zero-based indexing
        if len(relevant_indices) == 0:
            continue  # Skip if no relevant images found

        # Compute retrieval rank statistics
        GT = len(relevant_indices)  # Number of ground-truth relevant images
        R = min(GT, k)  # Effective number of relevant images considered
        r_avg = np.mean(relevant_indices[:R]) + 1  # Mean rank (convert to 1-based indexing)
        r_max = relevant_indices[R-1] + 1 if R > 0 else num_images  # Max rank (1-based)

        # Compute NMRR
        NMRR = (r_avg - (R + 1) / 2) / (r_max - (R + 1) / 2)
        ranks.append(NMRR)

    # Compute final ANMRR
    anmrr_score = np.mean(ranks)
    return anmrr_score

# Example usage
def load_features(model_name, save_dir):
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")
    return features, labels

save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

for model_name in models:
    features, labels = load_features(model_name, save_dir)
    distance_matrix = 1 - np.dot(features, features.T)  # Compute distance (1 - similarity)
    anmrr_score = compute_anmrr(labels, distance_matrix, k=10)
    print(f"ANMRR for {model_name.upper()}: {anmrr_score:.4f}")


ANMRR for ALEXNET: 0.3211
ANMRR for RESNET50: 0.3244
ANMRR for VGG16: 0.4799
ANMRR for VGG19: 0.4611
ANMRR for INCEPTION: 0.3326


for TOP_K = 100

In [ ]:
import numpy as np

def compute_anmrr(labels, distance_matrix, k=100):
    """
    Compute the Average Normalized Modified Retrieval Rank (ANMRR).

    Parameters:
        labels (numpy array): Array of ground truth labels.
        distance_matrix (numpy array): Precomputed distance matrix (higher = less similar).
        k (int): The number of top retrieved images to consider.

    Returns:
        float: ANMRR score (lower is better).
    """
    num_images = len(labels)
    ranks = []

    for i in range(num_images):
        # Get sorted indices (excluding the query itself)
        sorted_indices = np.argsort(distance_matrix[i])
        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude self-match

        # Retrieve the corresponding sorted labels
        sorted_labels = labels[sorted_indices]

        # Find ranks of relevant items
        relevant_indices = np.where(sorted_labels == labels[i])[0]  # Zero-based indexing
        if len(relevant_indices) == 0:
            continue  # Skip if no relevant images found

        # Compute retrieval rank statistics
        GT = len(relevant_indices)  # Number of ground-truth relevant images
        R = min(GT, k)  # Effective number of relevant images considered
        r_avg = np.mean(relevant_indices[:R]) + 1  # Mean rank (convert to 1-based indexing)
        r_max = relevant_indices[R-1] + 1 if R > 0 else num_images  # Max rank (1-based)

        # Compute NMRR
        NMRR = (r_avg - (R + 1) / 2) / (r_max - (R + 1) / 2)
        ranks.append(NMRR)

    # Compute final ANMRR
    anmrr_score = np.mean(ranks)
    return anmrr_score

# Example usage
def load_features(model_name, save_dir):
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")
    return features, labels

save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

for model_name in models:
    features, labels = load_features(model_name, save_dir)
    distance_matrix = 1 - np.dot(features, features.T)  # Compute distance (1 - similarity)
    anmrr_score = compute_anmrr(labels, distance_matrix, k=100)
    print(f"ANMRR for {model_name.upper()}: {anmrr_score:.4f}")


ANMRR for ALEXNET: 0.2363
ANMRR for RESNET50: 0.2367
ANMRR for VGG16: 0.3162
ANMRR for VGG19: 0.3193
ANMRR for INCEPTION: 0.2572


for all

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def compute_anmrr(labels, distance_matrix):
    num_images = len(labels)
    anmrr_values = []

    for i in range(num_images):
        # Sort indices based on distance (ascending order)
        sorted_indices = np.argsort(distance_matrix[i])
        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude self-match

        # Retrieve the sorted labels
        sorted_labels = labels[sorted_indices]

        # Identify relevant items
        relevant_items = (sorted_labels == labels[i])
        num_relevant = np.sum(relevant_items)

        if num_relevant == 0:
            anmrr_values.append(1)  # Worst case if no relevant images found
            continue

        # Compute rank positions of relevant items
        rank_positions = np.where(relevant_items)[0] + 1  # Convert to 1-based indexing

        # Compute Modified Retrieval Rank (MRR)
        mrr = np.sum(rank_positions) / num_relevant

        # Compute Normalized Modified Retrieval Rank (NMRR)
        K = num_images  # Since we are considering all images
        nmrr = (mrr - 0.5 * (1 + num_relevant)) / (1.25 * K - 0.5 * (1 + num_relevant))

        anmrr_values.append(nmrr)

    # Mean of NMRR values
    anmrr = np.mean(anmrr_values)
    return anmrr

# Directory where features are saved
save_dir = "/content/drive/MyDrive/NewONeFeature_Vectors"

# Models list
models = ["alexnet", "resnet50", "vgg16", "vgg19", "inception"]

# Compute and print ANMRR for each model (all images)
for model_name in models:
    features = np.load(f"{save_dir}/{model_name}_features.npy")
    labels = np.load(f"{save_dir}/{model_name}_labels.npy")

    # Compute distance matrix efficiently using cdist (Euclidean distance)
    distance_matrix = cdist(features, features, metric='euclidean')

    # Compute ANMRR for all images
    anmrr_score = compute_anmrr(labels, distance_matrix)
    print(f"ANMRR for {model_name.upper()} (All Images): {anmrr_score:.4f}")


ANMRR for ALEXNET (All Images): 0.1225
ANMRR for RESNET50 (All Images): 0.0865
ANMRR for VGG16 (All Images): 0.1648
ANMRR for VGG19 (All Images): 0.1613
ANMRR for INCEPTION (All Images): 0.1109


**PCA and IFK features vectors**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from scipy.stats import multivariate_normal
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=True)
conv5_2_layer = tf.keras.Model(inputs=base_model.input,
                               outputs=base_model.get_layer('block5_conv2').output)
fc6_layer = tf.keras.Model(inputs=base_model.input,
                           outputs=base_model.get_layer('fc1').output)

def extract_features(model, image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

def fisher_vector_aggregation(features):
    pca = PCA(n_components=32)
    transformed_features = pca.fit_transform(features)
    mean = np.mean(transformed_features, axis=0)
    cov = np.cov(transformed_features, rowvar=False)
    return multivariate_normal(mean=mean, cov=cov).rvs()

conv5_2_features, fc6_features = [], []
labels = []

categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                conv5_2_feat = extract_features(conv5_2_layer, img_path)
                fc6_feat = extract_features(fc6_layer, img_path)
                conv5_2_features.append(conv5_2_feat)
                fc6_features.append(fc6_feat)
                labels.append(idx)

# Apply IFK
conv5_2_features = fisher_vector_aggregation(np.array(conv5_2_features))
fc6_features = fisher_vector_aggregation(np.array(fc6_features))

# Save features
np.save(os.path.join(save_dir, "vgg16_conv5_2_features.npy"), conv5_2_features)
np.save(os.path.join(save_dir, "vgg16_fc6_features.npy"), fc6_features)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted and saved for VGG16 (conv5_2 and fc6)!")


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 25s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/st

/usr/local/lib/python3.11/dist-packages/scipy/stats/_multivariate.py:757: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  out = random_state.multivariate_normal(mean, cov, size)


✅ Features extracted and saved for VGG16 (conv5_2 and fc6)!


In [ ]:
conv5_2_features = np.vstack(conv5_2_features)  # Stack all batches
fc6_features = np.vstack(fc6_features)

print(conv5_2_features.shape)  # Expected: (2100, D1)
print(fc6_features.shape)      # Expected: (2100, 4096)



(32, 1)
(32, 1)


2nd code

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=True)
conv5_2_layer = tf.keras.Model(inputs=base_model.input,
                               outputs=base_model.get_layer('block5_conv2').output)
fc6_layer = tf.keras.Model(inputs=base_model.input,
                           outputs=base_model.get_layer('fc1').output)

# Function to extract features
def extract_features(model, img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
conv5_2_features, fc6_features, labels = [], [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                conv5_2_feat = extract_features(conv5_2_layer, img_path)
                fc6_feat = extract_features(fc6_layer, img_path)
                conv5_2_features.append(conv5_2_feat)
                fc6_features.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
conv5_2_features = np.array(conv5_2_features)
fc6_features = np.array(fc6_features)

# Apply PCA to reduce to (2100, 32)
pca_conv5_2 = PCA(n_components=32)
conv5_2_reduced = pca_conv5_2.fit_transform(conv5_2_features)

pca_fc6 = PCA(n_components=32)
fc6_reduced = pca_fc6.fit_transform(fc6_features)

# Save features as separate .npy files
np.save(os.path.join(save_dir, "vgg16_conv5_2_features.npy"), conv5_2_reduced)
np.save(os.path.join(save_dir, "vgg16_fc6_features.npy"), fc6_reduced)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted and saved successfully! (Shape: (2100, 32))")

✅ Features extracted and saved successfully! (Shape: (2100, 32))


In [ ]:
print(conv5_2_features.shape)  # Expected: (2100, 32)
print(fc6_features.shape)      # Expected: (2100 ,32)


(2100, 100352)
(2100, 4096)


In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensions to 32
pca_conv5_2 = PCA(n_components=32)
conv5_2_reduced = pca_conv5_2.fit_transform(conv5_2_features)
print(conv5_2_reduced.shape)  # Expected: (2100, 32

pca_fc6 = PCA(n_components=32)
fc6_reduced = pca_fc6.fit_transform(fc6_features)
print(fc6_reduced.shape)

(2100, 32)
(2100, 32)


In [ ]:
# Save reduced features
np.save(os.path.join(save_dir, "vgg16_conv5_2__32_features.npy"), conv5_2_reduced)
np.save(os.path.join(save_dir, "vgg16_fc6_32_features.npy"), fc6_reduced)

print("✅ Features extracted and saved successfully! (Shape: (2100, 32))")

✅ Features extracted and saved successfully! (Shape: (2100, 32))


MAP

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
features = np.load(f"{save_dir}/vgg16_conv5_2__32_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for VGG16: {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for VGG16: 51.99%


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
features = np.load(f"{save_dir}/vgg16_fc6_32_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for VGG16: {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for VGG16: 56.38%


TRY NEW ONE

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input
from scipy.stats import multivariate_normal
from sklearn.decomposition import PCA

def fisher_vector_aggregation(features):
    pca = PCA(n_components=32)
    transformed_features = pca.fit_transform(features)
    mean = np.mean(transformed_features, axis=0)
    cov = np.cov(transformed_features, rowvar=False)
    return multivariate_normal(mean=mean, cov=cov).rvs()

In [ ]:
# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
conv5_2_features = np.load(f"{save_dir}/vgg16_conv5_2_features.npy")
fc6_features = np.load(f"{save_dir}/vgg16_fc6_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

print(conv5_2_features.shape)  # Expected: (2100, 100352)
print(fc6_features.shape)      # Expected: (2100, 4096)

# Apply IFK
conv5_2_features = fisher_vector_aggregation(np.array(conv5_2_features))
fc6_features = fisher_vector_aggregation(np.array(fc6_features))

(2100, 32)
(2100, 32)


In [ ]:
print(conv5_2_features.shape)  # Expected: (2100, D1)
print(fc6_features.shape)      # Expected: (2100, 4096

(32,)
(32,)


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=True)

# Extracting features from 'conv5_block3_out' (Last convolutional block)
conv5_layer = tf.keras.Model(inputs=base_model.input,
                             outputs=base_model.get_layer('conv5_block3_out').output)

# Extracting features from 'avg_pool' (Instead of FC6, since ResNet50 does not have fully connected layers)
fc6_layer = tf.keras.Model(inputs=base_model.input,
                           outputs=base_model.get_layer('avg_pool').output)

# Function to extract features
def extract_features(model, img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
conv5_features, fc6_features, labels = [], [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                conv5_feat = extract_features(conv5_layer, img_path)
                fc6_feat = extract_features(fc6_layer, img_path)
                conv5_features.append(conv5_feat)
                fc6_features.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
conv5_features = np.array(conv5_features)
fc6_features = np.array(fc6_features)

# Apply PCA to reduce dimensions to (2100, 32)
pca_conv5 = PCA(n_components=32)
conv5_reduced = pca_conv5.fit_transform(conv5_features)

pca_fc6 = PCA(n_components=32)
fc6_reduced = pca_fc6.fit_transform(fc6_features)

# Save features as separate .npy files
np.save(os.path.join(save_dir, "resnet50_conv5_32_features.npy"), conv5_reduced)
np.save(os.path.join(save_dir, "resnet50_fc6_32_features.npy"), fc6_reduced)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted and saved successfully for ResNet50! (Shape: (2100, 32))")


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Features extracted and saved successfully for ResNet50! (Shape: (2100, 32))


In [ ]:
print(conv5_reduced.shape)  # Expected: (2100, 32)
print(fc6_reduced.shape)      # Expected: (2100, 32

(2100, 32)
(2100, 32)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
features = np.load(f"{save_dir}/resnet50_fc6_32_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for VGG16: {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for VGG16: 58.52%


vector of locally aggregated description (VLAD) on ResNet 50

In [ ]:
from sklearn.cluster import KMeans

def compute_vlad(features, num_clusters=64):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(features)
    cluster_centers = kmeans.cluster_centers_
    labels = kmeans.predict(features)

    vlad_vector = np.zeros((features.shape[0], num_clusters * features.shape[1]))

    for i, feature in enumerate(features):
        cluster = labels[i]
        residual = feature - cluster_centers[cluster]  # Compute residual
        vlad_vector[i, cluster * features.shape[1]:(cluster + 1) * features.shape[1]] = residual

    # Apply L2 normalization
    vlad_vector = vlad_vector / np.linalg.norm(vlad_vector, axis=1, keepdims=True)

    return vlad_vector


In [ ]:
# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
conv5_features  = np.load(f"{save_dir}/resnet50_conv5_32_features.npy")
fc6_features = np.load(f"{save_dir}/resnet50_fc6_32_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

In [ ]:
conv5_vlad = compute_vlad(conv5_features, num_clusters=64)
fc6_vlad = compute_vlad(fc6_features, num_clusters=64)

pca_conv5 = PCA(n_components=32).fit_transform(conv5_vlad)
pca_fc6 = PCA(n_components=32).fit_transform(fc6_vlad)


In [ ]:
print(pca_conv5.shape)  # Expected: (2100, 32)
print(pca_fc6.shape)      # Expected: (2100, 32

(2100, 32)
(2100, 32)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
# features = np.load(f"{save_dir}/resnet50_fc6_32_features.npy")
features = pca_fc6;
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for VGG16: {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for VGG16: 18.30%


Fisher vector(FV) On ResNet50

In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

# Load extracted ResNet50 features
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
features = np.load(f"{save_dir}/resnet50_fc6_32_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

# Function to compute Fisher Vector (FV)
def compute_fisher_vector(features, num_clusters=32):  # Increased clusters
    """
    Compute Fisher Vector encoding for given features using Gaussian Mixture Model (GMM).

    Args:
        features (numpy array): Input feature matrix (num_samples, feature_dim)
        num_clusters (int): Number of Gaussian clusters for GMM

    Returns:
        numpy array: Fisher Vector representation of the input features
    """
    gmm = GaussianMixture(n_components=num_clusters, covariance_type='diag', random_state=42)
    gmm.fit(features)  # Train GMM on extracted features

    # Compute soft assignments (posterior probabilities)
    fisher_vector = gmm.predict_proba(features)

    # Power normalization (signed square root)
    fisher_vector = np.sign(fisher_vector) * np.sqrt(np.abs(fisher_vector))

    # L2 normalization
    fisher_vector = fisher_vector / np.linalg.norm(fisher_vector, axis=1, keepdims=True)

    return fisher_vector

# Apply Fisher Vector encoding
fv_features = compute_fisher_vector(features, num_clusters=32)  # Increased clusters

# Apply PCA for dimensionality reduction (safe choice)
pca = PCA(n_components=32)  # Ensures output is (2100, 32)
fv_features_pca = pca.fit_transform(fv_features)

# Save Fisher Vector encoded features
np.save(f"{save_dir}/resnet50_fc6_fv_32_features.npy", fv_features_pca)

print("✅ Fisher Vector (FV) applied and saved successfully! (Shape: 2100, 32)")


✅ Fisher Vector (FV) applied and saved successfully! (Shape: 2100, 32)


In [ ]:
print(fv_features_pca.shape)  # Expected: (2100, 32

(2100, 32)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
# features = np.load(f"{save_dir}/resnet50_fc6_32_features.npy")
features = fv_features_pca;
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for VGG16: {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for VGG16: 52.42%


In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np

# Load extracted ResNet50 features
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
features = np.load(f"{save_dir}/resnet50_fc6_32_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

def compute_fv_optimized(features, num_clusters=32):
    gmm = GaussianMixture(n_components=num_clusters, covariance_type='full', random_state=42)
    gmm.fit(features)  # Train GMM on extracted features

    # Compute soft assignments (posterior probabilities)
    fisher_vector = gmm.predict_proba(features)

    # Power normalization (signed square root)
    fisher_vector = np.sign(fisher_vector) * np.sqrt(np.abs(fisher_vector))

    # L2 normalization
    fisher_vector = fisher_vector / np.linalg.norm(fisher_vector, axis=1, keepdims=True)

    return fisher_vector

# Apply Fisher Vector encoding
fv_features_optimized= compute_fv_optimized(features, num_clusters=32)

# Save FV features
np.save("resnet50_fc6_fv1_32_features_optimized.npy", fv_features_optimized)
print("✅ Optimized Fisher Vector applied!")


✅ Optimized Fisher Vector applied!


In [ ]:
print(fv_features_optimized.shape)  # Expected: (2100, 32

(2100, 32)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
# features = np.load(f"{save_dir}/resnet50_fc6_32_features.npy")
features = fv_features_optimized;
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-match
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for ResNet 50: {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for ResNet 50: 52.86%
